In [1]:
import numpy as np
import awkward as ak

metadata = ak.metadata_from_parquet(
    "https://zenodo.org/records/14537442/files/chicago-taxi.parquet"
)

In [2]:
metadata["form"].type.show()

var * {
    trip: {
        sec: ?float32,
        km: ?float32,
        begin: {
            lon: ?float64,
            lat: ?float64,
            time: ?datetime64[ms]
        },
        end: {
            lon: ?float64,
            lat: ?float64,
            time: ?datetime64[ms]
        },
        path: var * {
            londiff: float32,
            latdiff: float32
        }
    },
    payment: {
        fare: ?float32,
        tips: ?float32,
        total: ?float32,
        type: categorical[type=string]
    },
    company: categorical[type=string]
}


In [4]:
taxi = ak.from_parquet(
    "https://zenodo.org/records/14537442/files/chicago-taxi.parquet",
    row_groups=[0],
    columns=["trip.km", "trip.begin.l*", "trip.end.l*", "trip.path.*"],
)

In [5]:
taxi.trip

<Array [[{km: 0, begin: {...}, ...}, ...], ...] type='353 * var * ?{km: ?fl...'>

In [6]:
taxi_trip_lat = taxi.trip.begin.lat + taxi.trip.path.latdiff
taxi_trip_lon = taxi.trip.begin.lon + taxi.trip.path.londiff

In [7]:
taxi_trip_lat

<Array [[[41.9, 41.9], ..., [42, ...]], ...] type='353 * var * option[var *...'>

In [8]:
taxi[("trip", "path", "lat")] = taxi_trip_lat
taxi[("trip", "path", "lon")] = taxi_trip_lon

In [9]:
taxi.type.show()

353 * var * ?{
    trip: {
        km: ?float32,
        begin: {
            lon: ?float64,
            lat: ?float64
        },
        end: {
            lon: ?float64,
            lat: ?float64
        },
        path: var * {
            londiff: float32,
            latdiff: float32,
            lat: float64,
            lon: float64
        }
    }
}


In [11]:
trip = ak.flatten(taxi.trip, axis=1)
trip

<Array [{km: 0, begin: {...}, ...}, ..., {...}] type='1003517 * ?{km: ?floa...'>

In [12]:
ix_length = ak.argsort(trip.km, ascending=False)

In [13]:
trip_longest = trip[ix_length[:3]]
trip_longest

<Array [{km: 1.57e+03, begin: {...}, ...}, ...] type='3 * ?{km: ?float32, b...'>

In [14]:
lat_lon_taxi_75 = ak.concatenate(
    (trip_longest.path.lat[..., np.newaxis], trip_longest.path.lon[..., np.newaxis]),
    axis=-1,
)

In [15]:
lat_lon_taxi_75

<Array [[[41.8, -87.8], ..., [41.9, ...]], ...] type='3 * option[var * 2 * ...'>

In [17]:
import ipyleaflet as ipl

map_taxi_75 = ipl.Map(
    basemap=ipl.basemap_to_tiles(ipl.basemaps.CartoDB.Voyager, "2022-04-08"),
    center=(41.8921, -87.6623),
    zoom=11,
)
for route in lat_lon_taxi_75:
    path = ipl.AntPath(locations=route.to_list(), delay=1000)
    map_taxi_75.add_layer(path)
map_taxi_75

Map(center=[41.8921, -87.6623], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…